In [ ]:
!wget   http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2

--2022-03-08 17:50:15--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘shape_predictor_68_face_landmarks.dat.bz2’

shape_predictor_68_ 100%[===================>]  61.07M  75.0MB/s    in 0.8s    

2022-03-08 17:50:16 (75.0 MB/s) - ‘shape_predictor_68_face_landmarks.dat.bz2’ saved [64040097/64040097]



In [ ]:
!bunzip2 /content/shape_predictor_68_face_landmarks.dat.bz2

datFile =  "/content/shape_predictor_68_face_landmarks.dat"

In [ ]:
import cv2
import numpy as np
import dlib
from imutils import face_utils

In [ ]:
from google.colab.patches import cv2_imshow

In [ ]:
def distance(ptA,ptB):
	dist = np.linalg.norm(ptA - ptB)
	return dist

def eye_AR(a,b,c,d,e,f):
	up = distance(b,d) + distance(c,e)
	down = distance(a,f)
	ratio = up/(2.0*down)


	if(ratio>0.25):
		return 2
	elif(ratio>0.21 and ratio<=0.25):
		return 1
	else:
		return 0

In [ ]:
cap = cv2.VideoCapture('video1.mp4')  #enter '0' for start the camera of your computer

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(datFile)

#compteurs
sleep = 0
drowsy = 0
active = 0


while True:
  _, frame = cap.read()
  gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
  faces = detector(frame)
 
  for face in faces:
    x1 = face.left()
    y1 = face.top()
    x2 = face.right()
    y2 = face.bottom()
    
    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)

    landmarks = predictor(gray, face)
    landmarks = face_utils.shape_to_np(landmarks)

    left_eye = eye_AR(landmarks[36],landmarks[37],landmarks[38], landmarks[41], landmarks[40], landmarks[39])
    right_eye = eye_AR(landmarks[42],landmarks[43],landmarks[44], landmarks[47], landmarks[46], landmarks[45])
        

    if(left_eye==0 or right_eye==0):
      sleep+=1
      drowsy=0
      active=0
      if(sleep>6):
        status="Na3ss"
        color = (255,0,0)

    elif(left_eye==1 or right_eye==1):
      sleep=0
      active=0
      drowsy+=1
      if(drowsy>6):
        status="Fih n3ass"
        color = (0,0,255)

    else:
      drowsy=0
      sleep=0
      active+=1
      if(active>6):
        status="Fay9"
        color = (0,255,0)
        	
    cv2.putText(frame, status, (x1, y1-2), cv2.FONT_HERSHEY_SIMPLEX, 1.2, color,3)

    for n in range(0, 68):
      (x,y) = landmarks[n]
      cv2.circle(frame, (x, y), 1, (255, 255, 255), -1)

cv2_imshow(frame)